<a href="https://colab.research.google.com/github/YC0429/Customer_Revenue_Pred_Proj/blob/main/Google_Customer_Revenue_Pred_workplace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import json
import os
import datetime
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit, KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = None

#### Load the train dataset and transform the json columns into str

In [ ]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/Kaggle Related/Google revenue analysis/train.csv")
data.head()

<ipython-input-44-85e54dd921fc>:1: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western...",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472830385,1,1472830385
1,Organic Search,20160902,"{""browser"": ""Firefox"", ""browserVersion"": ""not ...",377306020877927890,"{""continent"": ""Oceania"", ""subContinent"": ""Aust...",377306020877927890_1472880147,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472880147,1,1472880147
2,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3895546263509774583,"{""continent"": ""Europe"", ""subContinent"": ""South...",3895546263509774583_1472865386,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472865386,1,1472865386
3,Organic Search,20160902,"{""browser"": ""UC Browser"", ""browserVersion"": ""n...",4763447161404445595,"{""continent"": ""Asia"", ""subContinent"": ""Southea...",4763447161404445595_1472881213,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472881213,1,1472881213
4,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",27294437909732085,"{""continent"": ""Europe"", ""subContinent"": ""North...",27294437909732085_1472822600,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1472822600,2,1472822600


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   channelGrouping       903653 non-null  object
 1   date                  903653 non-null  int64 
 2   device                903653 non-null  object
 3   fullVisitorId         903653 non-null  object
 4   geoNetwork            903653 non-null  object
 5   sessionId             903653 non-null  object
 6   socialEngagementType  903653 non-null  object
 7   totals                903653 non-null  object
 8   trafficSource         903653 non-null  object
 9   visitId               903653 non-null  int64 
 10  visitNumber           903653 non-null  int64 
 11  visitStartTime        903653 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 82.7+ MB


In [ ]:
json_col = ['device','geoNetwork','totals','trafficSource']
# You may need to use a loop if your DataFrame contains multiple rows with JSON data
json_df = []
for col in json_col:
  if len(json_df) == 0:
    json_df = pd.json_normalize(data[col].apply(json.loads))
  else:
    temp_df = pd.json_normalize(data[col].apply(json.loads))
    json_df = pd.concat([json_df,temp_df], axis=1)
json_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 47 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   browser                              903653 non-null  object
 1   browserVersion                       903653 non-null  object
 2   browserSize                          903653 non-null  object
 3   operatingSystem                      903653 non-null  object
 4   operatingSystemVersion               903653 non-null  object
 5   isMobile                             903653 non-null  bool  
 6   mobileDeviceBranding                 903653 non-null  object
 7   mobileDeviceModel                    903653 non-null  object
 8   mobileInputSelector                  903653 non-null  object
 9   mobileDeviceInfo                     903653 non-null  object
 10  mobileDeviceMarketingName            903653 non-null  object
 11  flashVersion              

In [ ]:
train = pd.concat([data.drop(json_col, axis=1), json_df], axis=1)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                               Non-Null Count   Dtype 
---  ------                               --------------   ----- 
 0   channelGrouping                      903653 non-null  object
 1   date                                 903653 non-null  int64 
 2   fullVisitorId                        903653 non-null  object
 3   sessionId                            903653 non-null  object
 4   socialEngagementType                 903653 non-null  object
 5   visitId                              903653 non-null  int64 
 6   visitNumber                          903653 non-null  int64 
 7   visitStartTime                       903653 non-null  int64 
 8   browser                              903653 non-null  object
 9   browserVersion                       903653 non-null  object
 10  browserSize                          903653 non-null  object
 11  operatingSystem           

In [ ]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,browserSize,operatingSystem,operatingSystemVersion,isMobile,mobileDeviceBranding,mobileDeviceModel,mobileInputSelector,mobileDeviceInfo,mobileDeviceMarketingName,flashVersion,language,screenColors,screenResolution,deviceCategory,continent,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation,visits,hits,pageviews,bounces,newVisits,transactionRevenue,campaign,source,medium,keyword,adwordsClickInfo.criteriaParameters,isTrueDirect,referralPath,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adwordsClickInfo.isVideoAd,adContent,campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,not available in demo dataset,ttnet.com.tr,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,not available in demo dataset,Macintosh,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,not available in demo dataset,unknown.unknown,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,1,1,1,1,NaN,(not set),google,organic,(not provided),not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,not available in demo dataset,not 

#### Do the same for the test data set

In [ ]:
data_test = pd.read_csv("drive/MyDrive/Colab Notebooks/Kaggle Related/Google revenue analysis/test.csv")

json_col = ['device','geoNetwork','totals','trafficSource']
# You may need to use a loop if your DataFrame contains multiple rows with JSON data
json_df = []
for col in json_col:
  if len(json_df) == 0:
    json_df = pd.json_normalize(data_test[col].apply(json.loads))
  else:
    temp_df = pd.json_normalize(data_test[col].apply(json.loads))
    json_df = pd.concat([json_df,temp_df], axis=1)
json_df.info()

test = pd.concat([data_test.drop(json_col, axis=1), json_df], axis=1)
test.info()

<ipython-input-49-e1f50cbc904e>:1: DtypeWarning:

Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.



In [ ]:
test['date'] = test['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
test.info()

### View transaction distribution

In [ ]:
train["transactionRevenue"] = train["transactionRevenue"].astype('float')
# gdf = train.groupby("country")["transactionRevenue"].sum().reset_index()
gdf = train.groupby("fullVisitorId")["transactionRevenue"].sum().reset_index()

plt.figure(figsize=(8,6))
plt.scatter(range(gdf.shape[0]), np.sort(np.log1p(gdf["transactionRevenue"].values)))
plt.xlabel('index', fontsize=12)
plt.ylabel('TransactionRevenue', fontsize=12)
plt.show()

#### Based on the plot, we could notice that most of the customers never spend money

In [ ]:
num_of_consumption_record = pd.notnull(train["transactionRevenue"]).sum()
num_of_consumers = (gdf["transactionRevenue"]>0).sum()

print("数据集中有{}条消费记录，约占数据集的{:.2f}%".format(num_of_consumption_record, num_of_consumption_record/train.shape[0] * 100))
print("数据集中有{}个消费者，约占数据集的{:.2f}%".format(num_of_consumers, num_of_consumers/gdf.shape[0] * 100))

In [ ]:
train['date'] = train['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
train['date'].head()

#### Check if there exist any column with only 1 constant value, if so we will drop them

In [ ]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
const_cols

In [ ]:
train.drop(const_cols, axis=1, inplace=True)
train.info()

#### Let's check the distribution of revenue under different browser, device category and device os

In [ ]:
def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

In [ ]:
# Device Browser
cnt_srs_brows = train.groupby('browser')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_brows.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_brows = cnt_srs_brows.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs_brows["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs_brows["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs_brows["mean"].head(10), 'rgba(50, 171, 96, 0.6)')

# Device Category
cnt_srs_cat = train.groupby('deviceCategory')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_cat.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_cat = cnt_srs_cat.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs_cat["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace5 = horizontal_bar_chart(cnt_srs_cat["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')
trace6 = horizontal_bar_chart(cnt_srs_cat["mean"].head(10), 'rgba(71, 58, 131, 0.8)')

# Operating system
cnt_srs_os = train.groupby('operatingSystem')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_os.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs_os.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs_os["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace8 = horizontal_bar_chart(cnt_srs_os["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')
trace9 = horizontal_bar_chart(cnt_srs_os["mean"].head(10),'rgba(246, 78, 139, 0.6)')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.04,
                          subplot_titles=["Device Browser - Total Count", "Device Browser - Non-zero Revenue Count", "Device Browser - Mean Revenue",
                                          "Device Category - Total Count",  "Device Category - Non-zero Revenue Count", "Device Category - Mean Revenue",
                                          "Device OS - Total Count", "Device OS - Non-zero Revenue Count", "Device OS - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
py.iplot(fig, filename='device-plots')

##### Inference
- Device browser distribution looks similar on both the count and count of non-zero revenue plots
- On the device category front, desktop seem to have higher percentage of non-zero revenue counts compared to mobile devices.
- In device operating system, though the number of counts is more from windows, the number of counts where revenue is not zero is more for Macintosh.
- Chrome OS also has higher percentage of non-zero revenue counts
- On the mobile OS side, iOS has more percentage of non-zero revenue counts compared to Android

#### Do some date exploration

In [ ]:
def scatter_plot(cnt_srs, color):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

In [ ]:
cnt_srs_dt = train.groupby('date')['transactionRevenue'].agg(['size', 'count'])
cnt_srs_dt.columns = ["count", "count of non-zero revenue"]
cnt_srs_dt = cnt_srs_dt.sort_index()

trace1 = scatter_plot(cnt_srs_dt["count"], 'red')
trace2 = scatter_plot(cnt_srs_dt["count of non-zero revenue"], 'blue')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["Date - Total Count", "Date - Non-zero Revenue count"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, paper_bgcolor='rgb(233,233,233)', title="Date Plots")
py.iplot(fig, filename='date-plots')

##### Inferences:

- We have data from 1 Aug, 2016 to 31 July, 2017 in our training dataset
- In Nov 2016, though there is an increase in the count of visitors, there is no increase in non-zero revenue counts during that time period (relative to the mean).

##### Check the date distribution within test dataset

In [ ]:
cnt_srs_dt_test = test.groupby('date')['fullVisitorId'].size()
cnt_srs_dt_test = cnt_srs_dt_test.sort_index()

trace = scatter_plot(cnt_srs_dt_test, 'red')

layout = go.Layout(
    height=400,
    width=800,
    paper_bgcolor='rgb(233,233,233)',
    title='Dates in Test set'
)


data = [trace]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename="ActivationDate")

##### In the test set, we have dates from 2 Aug, 2017 to 30 Apr, 2018. So there are no common dates between train and test set. So it might be a good idea to do time based validation for this dataset.

#### Geographic exploration

In [ ]:
# Continent
cnt_srs_conti = train.groupby('continent')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_conti.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_conti = cnt_srs_conti.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs_conti["count"].head(10), 'rgba(58, 71, 80, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs_conti["count of non-zero revenue"].head(10), 'rgba(58, 71, 80, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs_conti["mean"].head(10), 'rgba(58, 71, 80, 0.6)')

# Sub-continent
cnt_srs_sub = train.groupby('subContinent')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_sub.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_sub = cnt_srs_sub.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs_sub["count"], 'orange')
trace5 = horizontal_bar_chart(cnt_srs_sub["count of non-zero revenue"], 'orange')
trace6 = horizontal_bar_chart(cnt_srs_sub["mean"], 'orange')

# Network domain
cnt_srs_net = train.groupby('networkDomain')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_net.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_net = cnt_srs_net.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs_net["count"].head(10), 'blue')
trace8 = horizontal_bar_chart(cnt_srs_net["count of non-zero revenue"].head(10), 'blue')
trace9 = horizontal_bar_chart(cnt_srs_net["mean"].head(10), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15,
                          subplot_titles=["Continent - Count", "Continent - Non-zero Revenue Count", "Continent - Mean Revenue",
                                          "Sub Continent - Count",  "Sub Continent - Non-zero Revenue Count", "Sub Continent - Mean Revenue",
                                          "Network Domain - Count", "Network Domain - Non-zero Revenue Count", "Network Domain - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1500, width=1200, paper_bgcolor='rgb(233,233,233)', title="Geography Plots")
py.iplot(fig, filename='geo-plots')

##### Inferences:
- On the continent plot, we can see that America has both higher number of counts as well as highest number of counts where the revenue is non-zero
- Though Asia and Europe has high number of counts, the number of non-zero revenue counts from these continents are comparatively low.
- We can infer the first two points from the sub-continents plot too.
- If the network domain is "unknown.unknown" rather than "(not set)", then the number of counts with non-zero revenue tend to be lower.

#### Traffic Exploration

In [ ]:
# Source
cnt_srs_sourc = train.groupby('source')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_sourc.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_sourc = cnt_srs_sourc.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs_sourc["count"].head(10), 'green')
trace2 = horizontal_bar_chart(cnt_srs_sourc["count of non-zero revenue"].head(10), 'green')
trace3 = horizontal_bar_chart(cnt_srs_sourc["mean"].head(10), 'green')

# Sub-continent
cnt_srs_med = train.groupby('medium')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_med.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_med = cnt_srs_med.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs_med["count"], 'purple')
trace5 = horizontal_bar_chart(cnt_srs_med["count of non-zero revenue"], 'purple')
trace6 = horizontal_bar_chart(cnt_srs_med["mean"], 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15,
                          subplot_titles=["Traffic Source - Count", "Traffic Source - Non-zero Revenue Count", "Traffic Source - Mean Revenue",
                                          "Traffic Source Medium - Count",  "Traffic Source Medium - Non-zero Revenue Count", "Traffic Source Medium - Mean Revenue"
                                          ])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1000, width=1200, paper_bgcolor='rgb(233,233,233)', title="Traffic Source Plots")
py.iplot(fig, filename='traffic-source-plots')

##### Inferences:

- In the traffic source plot, though Youtube has high number of counts in the
dataset, the number of non-zero revenue counts are very less.
- Google plex has a high ratio of non-zero revenue count to total count in the traffic source plot.
- On the traffic source medium, "referral" has more number of non-zero revenue count compared to "organic" medium.

In [ ]:
# Page views
cnt_srs_view = train.groupby('pageviews')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_view.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_view = cnt_srs_view.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs_view["count"].head(60), 'green')
trace2 = horizontal_bar_chart(cnt_srs_view["count of non-zero revenue"].head(60), 'green')
trace3 = horizontal_bar_chart(cnt_srs_view["mean"].head(60), 'green')

# hits
cnt_srs_hits = train.groupby('hits')['transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs_hits.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs_hits = cnt_srs_hits.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs_hits["count"].head(60), 'purple')
trace5 = horizontal_bar_chart(cnt_srs_hits["count of non-zero revenue"].head(60), 'purple')
trace6 = horizontal_bar_chart(cnt_srs_hits["mean"].head(60), 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15,
                          subplot_titles=["Total Pageviews - Count", "Total Pageviews - Non-zero Revenue Count", "Total Pageviews - Mean Revenue",
                                          "Total Hits - Count",  "Total Hits - Non-zero Revenue Count", "Total Hits - Mean Revenue"
                                          ])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Visitor Profile Plots")
py.iplot(fig, filename='visitor-profile-plots')

##### Inferences:

- Both these variables look very predictive
- Count plot shows decreasing nature i.e. we have a very high total count for less number of hits and page views per visitor transaction and the overall count decreases when the number of hits per visitor transaction increases.
- On the other hand, we can clearly see that when the number of hits / pageviews per visitor transaction increases, we see that there is a high number of non-zero revenue counts.

##### ADF test to check whether the data is stationary

In [ ]:
from statsmodels.tsa.stattools import adfuller


result = adfuller(cnt_srs_dt["count"])
print('ADF Statistic for total count:', result[0])
print('p-value for total count:', result[1])
print('Critical Values for total count:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

result = adfuller(cnt_srs_dt["count of non-zero revenue"])
print('ADF Statistic for non-zero revenue count:', result[0])
print('p-value for non-zero revenue count:', result[1])
print('Critical Values for non-zero revenue count:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

#### **Baseline Model:**

Now let us build a baseline model on this dataset. Before we start building models, let us look at the variable names which are there in train dataset and not in test dataset.

In [ ]:
print("Variables not in test but in train : ", set(train.columns).difference(set(test.columns)))

So apart from target variable, there is one more variable "trafficSource.campaignCode" not present in test dataset. We need to remove this variable while building models. Also we can drop the constant variables which we got earlier.

Another thing is we can remove the "sessionId" as it is a unique identifier of the visit.

In [ ]:
const_cols

In [ ]:
cols_to_drop = const_cols + ['sessionId', 'visitId']

train = train.drop(["sessionId", 'visitId'], axis=1)
test = test.drop(cols_to_drop, axis=1)

KeyError: "['sessionId'] not found in axis"

Do feature engineering and create development and validation splits based on time to build the model. We can take the last two months as validation sample.

In [ ]:
# Impute 0 for missing target values
train["transactionRevenue"].fillna(0, inplace=True)
train_y = train["transactionRevenue"].values
train_id = train["fullVisitorId"].values
test_id = test["fullVisitorId"].values

In [ ]:
# Split into numerical and categorical columns
num_cols = ["hits","pageviews","visitNumber","visitStartTime",'bounces','newVisits']
cat_cols = ["channelGrouping", "browser",
        "deviceCategory", "operatingSystem",
        "city", "continent", "campaignCode", "isMobile",
        "country", "metro",
        "networkDomain", "region",
        "subContinent", "adContent",
        "adwordsClickInfo.adNetworkType",
        "adwordsClickInfo.gclId",
        "adwordsClickInfo.page",
        "adwordsClickInfo.slot", "campaign",
        "keyword", "medium",
        "referralPath", "source",
        'adwordsClickInfo.isVideoAd', 'isTrueDirect']


29

In [ ]:
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,browser,operatingSystem,isMobile,deviceCategory,continent,subContinent,country,region,metro,city,networkDomain,hits,pageviews,bounces,newVisits,transactionRevenue,campaign,source,medium,keyword,isTrueDirect,referralPath,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adwordsClickInfo.isVideoAd,adContent,campaignCode
0,Organic Search,2016-09-02,1131660440785968503,1472830385,1,1472830385,Chrome,Windows,False,desktop,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,ttnet.com.tr,1,1,1,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,2016-09-02,377306020877927890,1472880147,1,1472880147,Firefox,Macintosh,False,desktop,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au,1,1,1,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,2016-09-02,3895546263509774583,1472865386,1,1472865386,Chrome,Windows,False,desktop,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,unknown.unknown,1,1,1,1,NaN,(not set),google,organic,(not provided),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,2016-09-02,4763447161404445595,1472881213,1,1472881213,UC Browser,Linux,False,desktop,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1,1,1,1,NaN,(not set),google,organic,google + online,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,2016-09-02,27294437909732085,1472822600,2,1472822600,Chrome,Android,True,mobile,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown,1,1,1,NaN,NaN,(not set),google,organic,(not provided),True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 35 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   channelGrouping                 903653 non-null  object 
 1   date                            903653 non-null  object 
 2   fullVisitorId                   903653 non-null  object 
 3   visitId                         903653 non-null  int64  
 4   visitNumber                     903653 non-null  int64  
 5   visitStartTime                  903653 non-null  int64  
 6   browser                         903653 non-null  object 
 7   operatingSystem                 903653 non-null  object 
 8   isMobile                        903653 non-null  bool   
 9   deviceCategory                  903653 non-null  object 
 10  continent                       903653 non-null  object 
 11  subContinent                    903653 non-null  object 
 12  country         

In [ ]:
train.isnull().sum()

channelGrouping                        0
date                                   0
fullVisitorId                          0
visitId                                0
visitNumber                            0
visitStartTime                         0
browser                                0
operatingSystem                        0
isMobile                               0
deviceCategory                         0
continent                              0
subContinent                           0
country                                0
region                                 0
metro                                  0
city                                   0
networkDomain                          0
hits                                   0
pageviews                            100
bounces                           453023
newVisits                         200593
transactionRevenue                892138
campaign                               0
source                                 0
medium          

#### Fill in the null values for each column, based on their distribution

In [ ]:
print(train["pageviews"].unique())
train["pageviews"] = train["pageviews"].fillna(-1)

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '20', '19', '21', '25', '22',
       '26', '23', '24', '30', '27', '29', '28', '32', '31', '34', '33',
       '38', '35', '40', '36', '37', '39', '43', '45', '47', '52', '44',
       '50', '46', '49', '51', '56', '59', '71', '57', '82', '84', '92',
       '90', '41', '79', '138', '48', '69', '112', '42', '65', '60', '63',
       '68', '58', '74', '165', '53', '54', '55', '61', '70', '80', '78',
       '72', '188', '73', '62', '75', '86', '197', '148', '67', '102',
       '64', '103', '144', nan, '81', '119', '104', '128', '87', '111',
       '66', '83', '135', '94', '91', '99', '77', '118', '202', '127',
       '93', '106', '155', '117', '76', '89', '85', '143', '105', '115',
       '95', '110', '137', '120', '116', '208', '126', '400', '101',
       '108', '333', '97', '96', '98', '125', '113', '327', '154', '121',
       '183', '123', '141', '224', '88', '195', '343', '156

In [ ]:
train["bounces"].unique()
train["bounces"] = train["bounces"].fillna(-1)

array(['1', nan], dtype=object)

In [ ]:
train["newVisits"].unique()
train["newVisits"] = train["newVisits"].fillna(-1)

array(['1', nan], dtype=object)